In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [32]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [33]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [34]:
df.sample(5)

,Question,Answer
5089,What did the Mother say to her son when she sa...,Your Dad and I are getting a divorce.
53813,Who doesn't like a hot date?,A necrophiliac.
11240,What do you get when you mix Hitler with a dol...,Adolfphin
47206,Why are turn signals great workers?,"When they get tired and burn out, they work tw..."
29857,What's the name of West Virginia's version of ...,Gummy Bear


In [35]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [36]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,"Look, let me go back in there and face the peril?","You were, Sir Galahad, You were in terrible pe..."
1,Here. Behind you.,Where?
2,What do you call someone with a large head?,The headmaster.
3,I'm sorry I treated you mean today. You should...,"Like even today. I woke up this morning, you ..."
4,Which gospel contains Jesus' parable about the...,Math hue.


Tokenization

In [37]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [38]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs